This notebook is to scrape football data from fbref.com

Much of the scraping code is taken from this repository: https://github.com/chmartin/FBref_EPL.

Run the first cell and then the further cells to get data for whichever leagues you want.

All data is courtesy of StatsBomb via FBref. Find me on Twitter @pathaleee!


In [ ]:
!pip install selenium pandas numpy bs4
!pip install lxml --upgrade --force-reinstall


  Using cached lxml-6.0.2-cp312-cp312-manylinux_2_26_x86_64.manylinux_2_28_x86_64.whl.metadata (3.6 kB)
Using cached lxml-6.0.2-cp312-cp312-manylinux_2_26_x86_64.manylinux_2_28_x86_64.whl (5.3 MB)
  Attempting uninstall: lxml
    Found existing installation: lxml 6.0.2
    Uninstalling lxml-6.0.2:
      Successfully uninstalled lxml-6.0.2


In [ ]:
"""
#standard(stats)
stats = ["player","nationality","position","team","age","birth_year","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"]
stats3 = ["players_used","possession","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"] 
#goalkeeping(keepers)
keepers = ["player","nationality","position","team","age","birth_year","games_gk","games_starts_gk","minutes_gk","goals_against_gk","goals_against_per90_gk","shots_on_target_against","saves","save_pct","wins_gk","draws_gk","losses_gk","clean_sheets","clean_sheets_pct","pens_att_gk","pens_allowed","pens_saved","pens_missed_gk"]
keepers3 = ["players_used","games_gk","games_starts_gk","minutes_gk","goals_against_gk","goals_against_per90_gk","shots_on_target_against","saves","save_pct","wins_gk","draws_gk","losses_gk","clean_sheets","clean_sheets_pct","pens_att_gk","pens_allowed","pens_saved","pens_missed_gk"]
#advance goalkeeping(keepersadv)
keepersadv = ["player","nationality","position","team","age","birth_year","minutes_90s","goals_against_gk","pens_allowed","free_kick_goals_against_gk","corner_kick_goals_against_gk","own_goals_against_gk","psxg_gk","psnpxg_per_shot_on_target_against","psxg_net_gk","psxg_net_per90_gk","passes_completed_launched_gk","passes_launched_gk","passes_pct_launched_gk","passes_gk","passes_throws_gk","pct_passes_launched_gk","passes_length_avg_gk","goal_kicks","pct_goal_kicks_launched","goal_kick_length_avg","crosses_gk","crosses_stopped_gk","crosses_stopped_pct_gk","def_actions_outside_pen_area_gk","def_actions_outside_pen_area_per90_gk","avg_distance_def_actions_gk"]
keepersadv2 = ["minutes_90s","goals_against_gk","pens_allowed","free_kick_goals_against_gk","corner_kick_goals_against_gk","own_goals_against_gk","psxg_gk","psnpxg_per_shot_on_target_against","psxg_net_gk","psxg_net_per90_gk","passes_completed_launched_gk","passes_launched_gk","passes_pct_launched_gk","passes_gk","passes_throws_gk","pct_passes_launched_gk","passes_length_avg_gk","goal_kicks","pct_goal_kicks_launched","goal_kick_length_avg","crosses_gk","crosses_stopped_gk","crosses_stopped_pct_gk","def_actions_outside_pen_area_gk","def_actions_outside_pen_area_per90_gk","avg_distance_def_actions_gk"]
#shooting(shooting)
shooting = ["player","nationality","position","team","age","birth_year","minutes_90s","goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]
shooting2 = ["minutes_90s","goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]
shooting3 = ["goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]
#passing(passing)
passing = ["player","nationality","position","team","age","birth_year","minutes_90s","passes_completed","passes","passes_pct","passes_total_distance","passes_progressive_distance","passes_completed_short","passes_short","passes_pct_short","passes_completed_medium","passes_medium","passes_pct_medium","passes_completed_long","passes_long","passes_pct_long","assists","xa","xa_net","assisted_shots","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","progressive_passes"]
passing2 = ["passes_completed","passes","passes_pct","passes_total_distance","passes_progressive_distance","passes_completed_short","passes_short","passes_pct_short","passes_completed_medium","passes_medium","passes_pct_medium","passes_completed_long","passes_long","passes_pct_long","assists","xa","xa_net","assisted_shots","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","progressive_passes"]
#passtypes(passing_types)
passing_types = ["player","nationality","position","team","age","birth_year","minutes_90s","passes","passes_live","passes_dead","passes_free_kicks","through_balls","passes_pressure","passes_switches","crosses","corner_kicks","corner_kicks_in","corner_kicks_out","corner_kicks_straight","passes_ground","passes_low","passes_high","passes_left_foot","passes_right_foot","passes_head","throw_ins","passes_other_body","passes_completed","passes_offsides","passes_oob","passes_intercepted","passes_blocked"]
passing_types2 = ["passes","passes_live","passes_dead","passes_free_kicks","through_balls","passes_pressure","passes_switches","crosses","corner_kicks","corner_kicks_in","corner_kicks_out","corner_kicks_straight","passes_ground","passes_low","passes_high","passes_left_foot","passes_right_foot","passes_head","throw_ins","passes_other_body","passes_completed","passes_offsides","passes_oob","passes_intercepted","passes_blocked"]
#goal and shot creation(gca)
gca = ["player","nationality","position","team","age","birth_year","minutes_90s","sca","sca_per90","sca_passes_live","sca_passes_dead","sca_dribbles","sca_shots","sca_fouled","gca","gca_per90","gca_passes_live","gca_passes_dead","gca_dribbles","gca_shots","gca_fouled","gca_defense"]
gca2 = ["sca","sca_per90","sca_passes_live","sca_passes_dead","sca_dribbles","sca_shots","sca_fouled","gca","gca_per90","gca_passes_live","gca_passes_dead","gca_dribbles","gca_shots","gca_fouled","gca_defense"]
#defensive actions(defense)
defense = ["player","nationality","position","team","age","birth_year","minutes_90s","tackles","tackles_won","tackles_def_3rd","tackles_mid_3rd","tackles_att_3rd","dribble_tackles","dribbles_vs","dribble_tackles_pct","dribbled_past","pressures","pressure_regains","pressure_regain_pct","pressures_def_3rd","pressures_mid_3rd","pressures_att_3rd","blocks","blocked_shots","blocked_shots_saves","blocked_passes","interceptions","clearances","errors"]
defense2 = ["tackles","tackles_won","tackles_def_3rd","tackles_mid_3rd","tackles_att_3rd","dribble_tackles","dribbles_vs","dribble_tackles_pct","dribbled_past","pressures","pressure_regains","pressure_regain_pct","pressures_def_3rd","pressures_mid_3rd","pressures_att_3rd","blocks","blocked_shots","blocked_shots_saves","blocked_passes","interceptions","clearances","errors"]
#possession(possession)
possession = ["player","nationality","position","team","age","birth_year","minutes_90s","touches","touches_def_pen_area","touches_def_3rd","touches_mid_3rd","touches_att_3rd","touches_att_pen_area","touches_live_ball","dribbles_completed","dribbles","dribbles_completed_pct","players_dribbled_past","nutmegs","carries","carry_distance","carry_progressive_distance","progressive_carries","carries_into_final_third","carries_into_penalty_area","pass_targets","passes_received","passes_received_pct","miscontrols","dispossessed"]
possession2 = ["touches","touches_def_pen_area","touches_def_3rd","touches_mid_3rd","touches_att_3rd","touches_att_pen_area","touches_live_ball","dribbles_completed","dribbles","dribbles_completed_pct","players_dribbled_past","nutmegs","carries","carry_distance","carry_progressive_distance","progressive_carries","carries_into_final_third","carries_into_penalty_area","pass_targets","passes_received","passes_received_pct","miscontrols","dispossessed"]
#playingtime(playingtime)
playingtime = ["player","nationality","position","team","age","birth_year","minutes_90s","games","minutes","minutes_per_game","minutes_pct","games_starts","minutes_per_start","games_subs","minutes_per_sub","unused_subs","points_per_match","on_goals_for","on_goals_against","plus_minus","plus_minus_per90","plus_minus_wowy","on_xg_for","on_xg_against","xg_plus_minus","xg_plus_minus_per90","xg_plus_minus_wowy"]
playingtime2 = ["games","minutes","minutes_per_game","minutes_pct","games_starts","minutes_per_start","games_subs","minutes_per_sub","unused_subs","points_per_match","on_goals_for","on_goals_against","plus_minus","plus_minus_per90","plus_minus_wowy","on_xg_for","on_xg_against","xg_plus_minus","xg_plus_minus_per90","xg_plus_minus_wowy"]
#miscallaneous(misc)
misc = ["player","nationality","position","team","age","birth_year","minutes_90s","cards_yellow","cards_red","cards_yellow_red","fouls","fouled","offsides","crosses","interceptions","tackles_won","pens_won","pens_conceded","own_goals","ball_recoveries","aerials_won","aerials_lost","aerials_won_pct"]
misc2 = ["cards_yellow","cards_red","cards_yellow_red","fouls","fouled","offsides","crosses","interceptions","tackles_won","pens_won","pens_conceded","own_goals","ball_recoveries","aerials_won","aerials_lost","aerials_won_pct"]
"""

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import time
import pdb

# --- Selenium Setup ---
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                     "AppleWebKit/537.36 (KHTML, like Gecko) "
                     "Chrome/124.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)


# --- Core function using Selenium instead of requests ---
def get_tables(url, text):
    driver.get(url)
    time.sleep(3)  # give JS time to load
    html = driver.page_source
    print(f'getting table from : {url}')
    soup = BeautifulSoup(html, "html.parser")
    all_tables = soup.find_all("tbody")
    all_features = soup.find_all("thead")


    if len(all_tables) < 3 and len(all_features) < 3:
        raise ValueError("Could not find expected tables")

    team_table = all_tables[0]
    features_team_table = [th['data-stat']  for th in all_features[0].find_all("tr")[-1].find_all("th")]
    team_vs_table = all_tables[1]
    features_team_vs_table = [th['data-stat']  for th in all_features[1].find_all("tr")[-1].find_all("th")]
    player_table = all_tables[2]
    features_player_table = [th['data-stat']  for th in all_features[2].find_all("tr")[-1].find_all("th")]
    
    if text == "for":
        return player_table, team_table,features_player_table,features_team_table
    if text == "vs":
        return player_table, team_vs_table,features_player_table,features_team_vs_table
    driver.quit()



# --- Keep your helper functions (unchanged) ---
def get_frame(endpoint,features, player_table):
    print(f'getting features [{len(features)}] from {endpoint}')
    pre_df_player = dict()
    rows_player = player_table.find_all("tr")

    for row in rows_player:
        if row.find("th", {"scope": "row"}) is not None:
            for f in features:
                cell = row.find("td", {"data-stat": f})
                text = cell.text.strip() if cell else "0"
                if text == "":
                    text = "0"
                if f not in ["player","nationality","position","team","age","birth_year"]:
                    text = float(text.replace(",", "")) if text.replace(".", "", 1).isdigit() else text

                pre_df_player.setdefault(f, []).append(text)
    df_player = pd.DataFrame.from_dict(pre_df_player)
    return df_player


def get_frame_team(endpoint,features, team_table):
    features.remove('team')
    print(f'getting features [{len(features)}] from {endpoint}')
    pre_df_team = dict()
    rows_team = team_table.find_all("tr")
    for row in rows_team:
        if row.find("th", {"scope": "row"}) is not None:
            name = row.find("th", {"data-stat": "team"}).text.strip()
            pre_df_team.setdefault("team", []).append(name)
            for f in features:
                cell = row.find("td", {"data-stat": f})
                text = cell.text.strip() if cell else "0"
                if text == "":
                    text = "0"
                if f not in ["player","nationality","position","team","age","birth_year"]:
                    text = float(text.replace(",", "")) if text.replace(".", "", 1).isdigit() else text
                pre_df_team.setdefault(f, []).append(text)
    
    df_team = pd.DataFrame.from_dict(pre_df_team)
    #print(df_team.head(1))
    return df_team


# --- Category helper wrappers ---
def frame_for_category(category, top, end):
    url = f"{top}{category}{end}"
    player_table, _,features_player,_ = get_tables(url, "for")
    return get_frame(category+end,features_player, player_table)


def frame_for_category_team(category, top, end, text):
    url = f"{top}{category}{end}"
    _, team_table,_,features_team = get_tables(url, text)
    return get_frame_team(category+end,features_team, team_table)



def get_outfield_data(top, end):
    df1 = frame_for_category('stats',top,end)
    df2 = frame_for_category('shooting',top,end)
    df3 = frame_for_category('passing',top,end)
    df4 = frame_for_category('passing_types',top,end)
    df5 = frame_for_category('gca',top,end)
    df6 = frame_for_category('defense',top,end)
    df7 = frame_for_category('possession',top,end)
    df8 = frame_for_category('misc',top,end)
    df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    return df


#Function to get keeping and advance goalkeeping data
def get_keeper_data(top, end):
    df1 = frame_for_category('keepers', top, end)
    df2 = frame_for_category('keepersadv', top, end)
    df = pd.concat([df1, df2], axis=1)
    df = df.loc[:, ~df.columns.duplicated()]
    return df



#Function to get team-wise data accross all categories as mentioned above
def get_team_data(top, end, text):
    df1 = frame_for_category_team('stats', top, end, text)
    df2 = frame_for_category_team('keepers', top, end, text)
    df3 = frame_for_category_team('keepersadv', top, end, text)
    df4 = frame_for_category_team('shooting', top, end, text)
    df5 = frame_for_category_team('passing', top, end, text)
    df6 = frame_for_category_team('passing_types', top, end, text)
    df7 = frame_for_category_team('gca', top, end, text)
    df8 = frame_for_category_team('defense', top, end, text)
    df9 = frame_for_category_team('possession', top, end, text)
    df10 = frame_for_category_team('misc', top, end, text)
    df11 = frame_for_category_team('playingtime', top, end, text)
    df = pd.concat(
        [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10,df11],
        axis=1
    )
    df = df.loc[:,~df.columns.duplicated()]

    return df


In [ ]:
#This cell is to get the outfield player data for any competition

#Go to the 'Standard stats' page of the league
#For Premier League 2020/21, the link is this: https://fbref.com/en/comps/9/stats/Premier-League-Stats
#Remove the 'stats', and pass the first and third part of the link as parameters like below


PL_outfield_25_26 = get_outfield_data('https://fbref.com/en/comps/9/','/Premier-League-Stats')
PL_outfield_24_25 = get_outfield_data('https://fbref.com/en/comps/9/2024-2025/','/2024-2025-Premier-League-Stats')
PL_outfield_23_24 = get_outfield_data('https://fbref.com/en/comps/9/2023-2024/','/2023-2024-Premier-League-Stats')
PL_outfield_22_23 = get_outfield_data('https://fbref.com/en/comps/9/2022-2023/','/2022-2023-Premier-League-Stats')
PL_outfield_21_22 = get_outfield_data('https://fbref.com/en/comps/9/2021-2022/','/2021-2022-Premier-League-Stats')
PL_outfield_20_21 = get_outfield_data('https://fbref.com/en/comps/9/2020-2021/','/2020-2021-Premier-League-Stats')

#Save csv file to Desktop
PL_outfield_25_26.to_csv('../data/fbref/PL_keeper/PL_outfield_25_26.csv',index=False)
PL_outfield_24_25.to_csv('../data/fbref/PL_keeper/PL_outfield_24_25.csv',index=False)
PL_outfield_23_24.to_csv('../data/fbref/PL_keeper/PL_outfield_23_24.csv',index=False)
PL_outfield_22_23.to_csv('../data/fbref/PL_keeper/PL_outfield_22_23.csv',index=False)
PL_outfield_21_22.to_csv('../data/fbref/PL_keeper/PL_outfield_21_22.csv',index=False)
PL_outfield_20_21.to_csv('../data/fbref/PL_keeper/PL_outfield_20_21.csv',index=False)



getting table from : https://fbref.com/en/comps/9/stats/Premier-League-Stats
getting features [37] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/shooting/Premier-League-Stats
getting features [26] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/passing/Premier-League-Stats
getting features [32] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/passing_types/Premier-League-Stats
getting features [24] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/gca/Premier-League-Stats
getting features [25] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/defense/Premier-League-Stats
getting features [25] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/possession/Premier-League-Stats
getting features [31] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/misc/Premier-League-Stats
getting features [25] fr

In [ ]:
#This cell is to get the goalkeeper data for any competition

#Go to the 'Standard stats' page of the league
#For Premier League 2020/21, the link is this: https://fbref.com/en/comps/9/stats/Premier-League-Stats
#Remove the 'stats', and pass the first and third part of the link as parameters like below
PL_keeper_25_26 = get_keeper_data('https://fbref.com/en/comps/9/','/Premier-League-Stats')
PL_keeper_24_25 = get_keeper_data('https://fbref.com/en/comps/9/2024-2025/','/2024-2025-Premier-League-Stats')
PL_keeper_23_24 = get_keeper_data('https://fbref.com/en/comps/9/2023-2024/','/2023-2024-Premier-League-Stats')
PL_keeper_22_23 = get_keeper_data('https://fbref.com/en/comps/9/2022-2023/','/2022-2023-Premier-League-Stats')
PL_keeper_21_22 = get_keeper_data('https://fbref.com/en/comps/9/2021-2022/','/2021-2022-Premier-League-Stats')
PL_keeper_20_21 = get_keeper_data('https://fbref.com/en/comps/9/2020-2021/','/2020-2021-Premier-League-Stats')

#Save csv file to Desktop
PL_keeper_25_26.to_csv('../data/fbref/PL_keeper/PL_keeper_25_26.csv',index=False)
PL_keeper_24_25.to_csv('../data/fbref/PL_keeper/PL_keeper_24_25.csv',index=False)
PL_keeper_23_24.to_csv('../data/fbref/PL_keeper/PL_keeper_23_24.csv',index=False)
PL_keeper_22_23.to_csv('../data/fbref/PL_keeper/PL_keeper_22_23.csv',index=False)
PL_keeper_21_22.to_csv('../data/fbref/PL_keeper/PL_keeper_21_22.csv',index=False)
PL_keeper_20_21.to_csv('../data/fbref/PL_keeper/PL_keeper_20_21.csv',index=False)

getting table from : https://fbref.com/en/comps/9/keepers/Premier-League-Stats
getting features [27] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/keepersadv/Premier-League-Stats
getting features [34] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/2024-2025/keepers/2024-2025-Premier-League-Stats
getting features [27] from /2024-2025-Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/2024-2025/keepersadv/2024-2025-Premier-League-Stats
getting features [34] from /2024-2025-Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/2023-2024/keepers/2023-2024-Premier-League-Stats
getting features [27] from /2023-2024-Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/2023-2024/keepersadv/2023-2024-Premier-League-Stats
getting features [34] from /2023-2024-Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/2022-2023/keepers/2022-2023-Premier-League-Stats
gett

In [ ]:
#This cell is to get the data FOR all teams in any competition

#Go to the 'Standard stats' page of the league
#For Premier League 2020/21, the link is this: https://fbref.com/en/comps/9/stats/Premier-League-Stats
#Remove the 'stats', and pass the first and third part of the link as parameters like below
#df_team = get_team_data('https://fbref.com/en/comps/9/','/Premier-League-Stats','for')

PL_team_25_26 = get_team_data('https://fbref.com/en/comps/9/','/Premier-League-Stats','for')
PL_team_24_25 = get_team_data('https://fbref.com/en/comps/9/2024-2025/','/2024-2025-Premier-League-Stats','for')
PL_team_23_24 = get_team_data('https://fbref.com/en/comps/9/2023-2024/','/2023-2024-Premier-League-Stats','for')
PL_team_22_23 = get_team_data('https://fbref.com/en/comps/9/2022-2023/','/2022-2023-Premier-League-Stats','for')
PL_team_21_22 = get_team_data('https://fbref.com/en/comps/9/2021-2022/','/2021-2022-Premier-League-Stats','for')
PL_team_20_21 = get_team_data('https://fbref.com/en/comps/9/2020-2021/','/2020-2021-Premier-League-Stats','for')

#Save csv file to Desktop
PL_team_25_26.to_csv('../data/fbref/PL_keeper/PL_team_25_26.csv',index=False)
PL_team_24_25.to_csv('../data/fbref/PL_keeper/PL_team_24_25.csv',index=False)
PL_team_23_24.to_csv('../data/fbref/PL_keeper/PL_team_23_24.csv',index=False)
PL_team_22_23.to_csv('../data/fbref/PL_keeper/PL_team_22_23.csv',index=False)
PL_team_21_22.to_csv('../data/fbref/PL_keeper/PL_team_21_22.csv',index=False)
PL_team_20_21.to_csv('../data/fbref/PL_keeper/PL_team_20_21.csv',index=False)

getting table from : https://fbref.com/en/comps/9/stats/Premier-League-Stats
getting features [31] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/keepers/Premier-League-Stats
getting features [20] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/keepersadv/Premier-League-Stats
getting features [27] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/shooting/Premier-League-Stats
getting features [19] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/passing/Premier-League-Stats
getting features [25] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/passing_types/Premier-League-Stats
getting features [17] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/gca/Premier-League-Stats
getting features [18] from /Premier-League-Stats
getting table from : https://fbref.com/en/comps/9/defense/Premier-League-Stats
getting features [18]

In [1]:
!pip install ydata_profiling

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport

df = pd.read_csv('../data/fbref/PL_outfield/PL_outfield_25_26.csv')
profile = ProfileReport(df, minimal=True, lazy=True)
profile.to_file("../data/comparison.html")

/home/kheaw/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 107.84it/s]


In [44]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 5.5 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [python-Levenshtein]


In [6]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process
import os

# --- Display setup ---
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# --- Paths ---
data_path = "../data/fbref/PL_outfield/"
save_path = "../data/fbref/PL_outfield/modified"
os.makedirs(save_path, exist_ok=True)

# --- Preload ranking data once ---
rankingdf = pd.read_csv('../data/UEFA_Ranking_2026.csv')
rankingdf['used_point'] = np.where(
    rankingdf['Total Points'] >= rankingdf['Country Part'],
    rankingdf['Total Points'],
    rankingdf['Country Part']
)
choices = rankingdf['team'].unique()

# --- Matching function ---
def match_team(name):
    if pd.isna(name): 
        return np.nan
    match, score = process.extractOne(name, choices)
    if score > 70:
        return rankingdf.loc[rankingdf['team'] == match, 'used_point'].iloc[0]
    return 0

# --- Outfield Players ---
outfield_files = [f for f in os.listdir(data_path) if "_outfield" in f and f.endswith(".csv")]

for file in outfield_files:
    print(f"⚙️ Processing: {file}")
    df = pd.read_csv(os.path.join(data_path, file))

    #drop cols
    if 'ranker' in df.columns:
        df.drop(columns=['ranker'], inplace=True)
    else: print(f'ranker doesnt exist in {file}')

    # Encode positions
    df_encoded = df.copy()
    if 'position' in df.columns:
        df_encoded = (
            df_encoded
            .assign(
                position_fw=df['position'].str.contains('FW', na=False).astype(int),
                position_mf=df['position'].str.contains('MF', na=False).astype(int),
                position_df=df['position'].str.contains('DF', na=False).astype(int),
            )
            .drop(columns=['position'])
        )
    else: print(f'position doesnt exist in {file}')

    #Trim age
    if df['age'].dtype == 'object':
        df_encoded['age'] = df_encoded['age'].str.split('-').str[0].str.strip()

    # Map UEFA coefficient
    df_encoded['team_coeff'] = df_encoded['team'].apply(match_team)
    #df_encoded.drop(columns=['team'], inplace=True)

    # Save cleaned file
    output_file = os.path.join(save_path, file)
    df_encoded.to_csv(output_file, index=False)
    print(f"Saved to: {output_file}")

⚙️ Processing: PL_outfield_21_22.csv
Saved to: ../data/fbref/PL_outfield/modified/PL_outfield_21_22.csv
⚙️ Processing: PL_outfield_23_24.csv
Saved to: ../data/fbref/PL_outfield/modified/PL_outfield_23_24.csv
⚙️ Processing: PL_outfield_22_23.csv
Saved to: ../data/fbref/PL_outfield/modified/PL_outfield_22_23.csv
⚙️ Processing: PL_outfield_24_25.csv
Saved to: ../data/fbref/PL_outfield/modified/PL_outfield_24_25.csv
⚙️ Processing: PL_outfield_20_21.csv
Saved to: ../data/fbref/PL_outfield/modified/PL_outfield_20_21.csv
⚙️ Processing: PL_outfield_25_26.csv
Saved to: ../data/fbref/PL_outfield/modified/PL_outfield_25_26.csv


In [ ]:
df_encoded['carry_progressive_distance']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
445    0.0
446    0.0
447    0.0
448    0.0
449    0.0
Name: carry_progressive_distance, Length: 450, dtype: float64